In [44]:
# read T5 scores for each seg
import collections
import math
from tqdm import tqdm
#score is dictionary of qid, docsegid, t5score
scores = collections.defaultdict(dict)
with open('../doctemp/d2qseg.dev.tsv') as f_query_run_ids, open(
        '../doctemp/mono_preds_d2qseg.dev.t5') as f_pred:
    for line_query_passage_id, line_pred in tqdm(zip(f_query_run_ids, f_pred)):
        query_id, passage_id, _ = line_query_passage_id.strip().split('\t')
        docseg_id = passage_id.split('.')[0]
        _, score = line_pred.strip().split('\t')
        score = float(score)
        scores[query_id][docseg_id] = score

51893214it [02:03, 420052.93it/s]


In [ ]:
with open('./missing.dev.tsv') as fin:
    for line in tqdm(fin):
        query_id, docseg_id, score, _ = line.split('\t')
        score = float(score)
        scores[query_id][docseg_id] = score

3020it [00:00, 8514.10it/s]


In [ ]:
ltr_k  = 750å
k = 1000
counts = None
counts = collections.defaultdict(int)
with open(f'run.ltr.doc.{ltr_k}from{k}.d2q_field.tsv', 'w') as output_f:
    with open(f'run.ltr.doc.{k}.d2q_field.tsv') as input_f:
        for line in tqdm(input_f):
            cols = line.split('\t')
            qid = cols[0]
            if(counts[qid] < ltr_k):
                docid = cols[2]
                counts[qid] += 1
                output_f.write(f'{qid}\t{docid}\t{counts[qid]}\n')

5191965it [00:09, 574505.58it/s]


In [ ]:
examples = None
examples = collections.defaultdict(dict)
with open(f'./run.ltr.doc.1000.d2q_field.tsv') as fcan, open(f'./run.monoT5.doc.ltr.1000.d2q_field.tsv', 'w') as fout:
    for line_query_passage_id in tqdm(fcan):
        query_id, _, passage_id, _, _, _= line_query_passage_id.strip().split('\t')
        docseg_id = passage_id.split('.')[0]
        doc_id = docseg_id.split('#')[0]
        if docseg_id in scores[query_id]:
            score = scores[query_id][docseg_id]
            score = float(score)
            if doc_id not in examples[query_id]:
                examples[query_id][doc_id] = [score]
            else:
                examples[query_id][doc_id].append(score)
            #examples is qid, doc_id with array of scores
    
    for query_id, doc_ids_scores in tqdm(examples.items()):
        #by taking maxP to get doc score
        doc_ids_scores = [
            (doc_id, max(scores))
            for doc_id, scores in doc_ids_scores.items()]

        doc_ids_scores.sort(key=lambda x: x[1], reverse=True)
        rank = 1
        for doc_id, score in doc_ids_scores:
            score = math.exp(score)
            # Adds small score based on rank to avoid duplicated scores.
            score += 1e-6 / rank
            fout.write(f'{query_id}\t{doc_id}\t{rank}\n')
            rank += 1

5191965it [00:17, 296145.12it/s]
100%|██████████| 5193/5193 [00:03<00:00, 1504.73it/s]


In [ ]:
import os
for ltr_k in [900]:
    for k in [1000]:
        stream = os.popen(f"python3 ../RA/pyserini/tools/scripts/msmarco/msmarco_doc_eval.py \
         --judgments ../RA/pyserini/tools/topics-and-qrels/qrels.msmarco-doc.dev.txt --run ./run.monoT5.doc.ltr.{ltr_k}from{k}.d2q_field.tsv")
        output = stream.read()
        outputs = output.split('Quantity of Documents ranked for each query is as expected.')
        print(f'{ltr_k} from {k}:')
        print(outputs[1])

900 from 1000:
 Evaluating
#####################
MRR @100: 0.4050544026219841
QueriesRanked: 5193
#####################



In [ ]:
!python3 ../RA/pyserini/tools/scripts/msmarco/convert_msmarco_to_trec_run.py --input ./run.monoT5.doc.ltr.750from1000.d2q_field.tsv --output ./run.monoT5.doc.ltr.750from1000.d2q_field.trec

Done!


In [ ]:
!../RA/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mmap -m ndcg_cut.10\
 ../RA/pyserini/tools/topics-and-qrels/qrels.msmarco-doc.dev.txt ./run.monoT5.doc.ltr.750from1000.d2q_field.trec

map                   	all	0.4054
ndcg_cut_10           	all	0.4702


In [ ]:
for ltr_k in [100,200,300,700,800,900,1000]:
    for k in [1000]:
        counts = None
        counts = collections.defaultdict(int)
        with open(f'run.ltr.doc.{ltr_k}from{k}.base_new.tsv', 'w') as output_f:
            with open(f'run.ltr.doc.{k}.base_new.txt') as input_f:
                for line in tqdm(input_f):
                    cols = line.split('\t')
                    qid = cols[0]
                    if(counts[qid] < ltr_k):
                        docid = cols[2]
                        counts[qid] += 1
                        output_f.write(f'{qid}\t{docid}\t{counts[qid]}\n')

5191965it [00:07, 731274.33it/s]
5191965it [00:07, 695534.75it/s]
5191965it [00:08, 605442.03it/s]
5191965it [00:13, 381455.48it/s]
5191965it [00:10, 479409.24it/s]
5191965it [00:10, 500090.06it/s]
5191965it [00:10, 488116.75it/s]


In [ ]:
for ltr_k in [100,200,300,700,800,900,1000]:
    for k in [1000]:
        examples = None
        examples = collections.defaultdict(dict)
        with open(f'./run.ltr.doc.{ltr_k}from{k}.base_new.tsv') as fcan, open(f'./run.monoT5.doc.ltr.{ltr_k}from{k}.base_new.tsv', 'w') as fout:
            for line_query_passage_id in tqdm(fcan):
                query_id, passage_id, _= line_query_passage_id.strip().split('\t')
                docseg_id = passage_id.split('.')[0]
                doc_id = docseg_id.split('#')[0]
                if docseg_id in scores[query_id]:
                    score = scores[query_id][docseg_id]
                    score = float(score)
                    if doc_id not in examples[query_id]:
                        examples[query_id][doc_id] = [score]
                    else:
                        examples[query_id][doc_id].append(score)
            #examples is qid, doc_id with array of scores
    
            for query_id, doc_ids_scores in tqdm(examples.items()):
                #by taking maxP to get doc score
                doc_ids_scores = [
                    (doc_id, max(scores))
                    for doc_id, scores in doc_ids_scores.items()]

                doc_ids_scores.sort(key=lambda x: x[1], reverse=True)
                rank = 1
                for doc_id, score in doc_ids_scores:
                    score = math.exp(score)
                    # Adds small score based on rank to avoid duplicated scores.
                    score += 1e-6 / rank
                    fout.write(f'{query_id}\t{doc_id}\t{rank}\n')
                    rank += 1

519300it [00:08, 64210.28it/s] 
100%|██████████| 5193/5193 [00:01<00:00, 4481.58it/s]
1038600it [00:07, 139380.21it/s]
100%|██████████| 5193/5193 [00:00<00:00, 5265.21it/s]
1557900it [00:12, 120401.07it/s]
100%|██████████| 5193/5193 [00:01<00:00, 3484.05it/s]
3634665it [00:29, 122114.56it/s]
100%|██████████| 5193/5193 [00:03<00:00, 1426.25it/s]
4153765it [00:33, 125258.29it/s]
100%|██████████| 5193/5193 [00:07<00:00, 731.76it/s] 
4672865it [00:36, 128361.85it/s]
100%|██████████| 5193/5193 [00:06<00:00, 830.91it/s] 
5191965it [00:38, 136127.41it/s]
100%|██████████| 5193/5193 [00:06<00:00, 864.26it/s] 


In [ ]:
import os
for ltr_k in [100,200,300,400,500,600,700,800,900,1000]:
    for k in [1000]:
        stream = os.popen(f"python3 ../RA/pyserini/tools/scripts/msmarco/msmarco_doc_eval.py \
         --judgments ../RA/pyserini/tools/topics-and-qrels/qrels.msmarco-doc.dev.txt --run ./run.monoT5.doc.ltr.{ltr_k}from{k}.base_new.tsv")
        output = stream.read()
        outputs = output.split('Quantity of Documents ranked for each query is as expected.')
        print(f'{ltr_k} from {k}:')
        print(outputs[1])

100 from 1000:
 Evaluating
#####################
MRR @100: 0.39720802538941846
QueriesRanked: 5193
#####################

200 from 1000:
 Evaluating
#####################
MRR @100: 0.4017577761708004
QueriesRanked: 5193
#####################

300 from 1000:
 Evaluating
#####################
MRR @100: 0.40307662966106717
QueriesRanked: 5193
#####################

400 from 1000:
 Evaluating
#####################
MRR @100: 0.4043899355697052
QueriesRanked: 5193
#####################

500 from 1000:
 Evaluating
#####################
MRR @100: 0.40505063985305223
QueriesRanked: 5193
#####################

600 from 1000:
 Evaluating
#####################
MRR @100: 0.40522863975785184
QueriesRanked: 5193
#####################

700 from 1000:
 Evaluating
#####################
MRR @100: 0.40571715081876386
QueriesRanked: 5193
#####################

800 from 1000:
 Evaluating
#####################
MRR @100: 0.4055788897397871
QueriesRanked: 5193
#####################

900 from 1000:
 Evaluating


In [ ]:
import os
for ltr_k in [100,200,300,400,500,600,700,800,900,1000]:
    for k in [1000]:
        stream = os.popen(f"python3 ../RA/pyserini/tools/scripts/msmarco/msmarco_doc_eval.py \
         --judgments ../RA/pyserini/tools/topics-and-qrels/qrels.msmarco-doc.dev.txt --run ./run.monoT5.doc.ltr.{ltr_k}from{k}.d2q_field.tsv")
        output = stream.read()
        outputs = output.split('Quantity of Documents ranked for each query is as expected.')
        print(f'{ltr_k} from {k}:')
        print(outputs[1])

100 from 1000:
 Evaluating
#####################
MRR @100: 0.3841586771194696
QueriesRanked: 5193
#####################

200 from 1000:
 Evaluating
#####################
MRR @100: 0.3943694863566146
QueriesRanked: 5193
#####################

300 from 1000:
 Evaluating
#####################
MRR @100: 0.39987412284065704
QueriesRanked: 5193
#####################

400 from 1000:
 Evaluating
#####################
MRR @100: 0.40125259823668014
QueriesRanked: 5193
#####################

500 from 1000:
 Evaluating
#####################
MRR @100: 0.40274899390213503
QueriesRanked: 5193
#####################

600 from 1000:
 Evaluating
#####################
MRR @100: 0.4044323946726725
QueriesRanked: 5193
#####################

700 from 1000:
 Evaluating
#####################
MRR @100: 0.4052316547886764
QueriesRanked: 5193
#####################

800 from 1000:
 Evaluating
#####################
MRR @100: 0.4055043206246541
QueriesRanked: 5193
#####################

900 from 1000:


IndexError: list index out of range

In [ ]:
!python3 ../RA/pyserini/tools/scripts/msmarco/convert_msmarco_to_trec_run.py --input ./run.monoT5.doc.ltr.550from1000.base.tsv --output ./run.monoT5.doc.ltr.550from1000.base.trec

Done!


In [ ]:
!../RA/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mmap -m ndcg_cut.10\
 ../RA/pyserini/tools/topics-and-qrels/qrels.msmarco-doc.dev.txt ./run.monoT5.doc.ltr.550from1000.base.trec

map                   	all	0.4052
ndcg_cut_10           	all	0.4699


In [ ]:
!python3 ../RA/pyserini/tools/scripts/msmarco/msmarco_doc_eval.py \
         --judgments ../RA/pyserini/tools/topics-and-qrels/qrels.msmarco-doc.dev.txt --run ./run.monoT5.doc.ltr.1000.d2q_field.tsv

Too many documents ranked. Please Provide top 100 documents for qid:1000000
Too many documents ranked. Please Provide top 100 documents for qid:1000004
Too many documents ranked. Please Provide top 100 documents for qid:1000006
Too many documents ranked. Please Provide top 100 documents for qid:1000017
Too many documents ranked. Please Provide top 100 documents for qid:1000030
Too many documents ranked. Please Provide top 100 documents for qid:1000083
Too many documents ranked. Please Provide top 100 documents for qid:1000097
Too many documents ranked. Please Provide top 100 documents for qid:100013
Too many documents ranked. Please Provide top 100 documents for qid:1000272
Too many documents ranked. Please Provide top 100 documents for qid:1000319
Too many documents ranked. Please Provide top 100 documents for qid:1000459
Too many documents ranked. Please Provide top 100 documents for qid:1000519
Too many documents ranked. Please Provide top 100 documents for qid:1000574
Too many docu

Too many documents ranked. Please Provide top 100 documents for qid:559018
Too many documents ranked. Please Provide top 100 documents for qid:559198
Too many documents ranked. Please Provide top 100 documents for qid:559507
Too many documents ranked. Please Provide top 100 documents for qid:559607
Too many documents ranked. Please Provide top 100 documents for qid:559709
Too many documents ranked. Please Provide top 100 documents for qid:559771
Too many documents ranked. Please Provide top 100 documents for qid:559959
Too many documents ranked. Please Provide top 100 documents for qid:560059
Too many documents ranked. Please Provide top 100 documents for qid:560245
Too many documents ranked. Please Provide top 100 documents for qid:56033
Too many documents ranked. Please Provide top 100 documents for qid:560357
Too many documents ranked. Please Provide top 100 documents for qid:560419
Too many documents ranked. Please Provide top 100 documents for qid:560673
Too many documents ranked.

#####################
MRR @100: 0.4053131224747211
QueriesRanked: 5193
#####################
